## Library Imports

In [2]:
# Cell 1
import numpy as np
import pandas as pd
import glob
import os
import logging
import tensorflow as tf
from tqdm import tqdm
from datetime import datetime
from scipy.stats import skew, kurtosis
from tensorflow.keras import layers, Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import LSTM, Dense



# Pre-processing of files 

In [3]:
# Cell 2
def load_emothaw_data(directory_path):
    all_data = []
    all_labels = []

    # Use glob to find all .svc files in the specified directory
    file_paths = glob.glob(os.path.join(directory_path, "*.svc"))
    if not file_paths:
        # Raise an error if no files are found
        raise ValueError("No files found in the specified directory")

    # Iterate over each file path found
    for file_path in file_paths:
        print(f"Processing file: {file_path}")
        try:
            # Extract the label from the filename
            # Assuming the label is the part before the first underscore '_'
            label = int(os.path.basename(file_path).split('_')[0])

            # Load the file and strip trailing spaces
            with open(file_path, 'r') as f:
                lines = [line.strip() for line in f.readlines() if line.strip()]  # Ignore empty lines

            # Convert the lines into a DataFrame by splitting each line by whitespace
            df = pd.DataFrame([line.split() for line in lines])  # Split by whitespace

            # Skip empty files
            if df.empty:
                print(f"Warning: {file_path} is empty after processing. Skipping file.")
                continue

            # First row contains the total number of data rows (excluding the first row)
            total_rows = int(df.iloc[0, 0])  # Number of expected sequences

            # Extract the feature data from the subsequent rows
            data = df.iloc[1:, :].values  # All columns are treated as features

            # Check if enough data rows exist as per the first row's instruction
            if data.shape[0] < total_rows:
                raise ValueError(f"Insufficient data: expected {total_rows}, but found {data.shape[0]}")

            # Ensure each row has exactly 7 feature values
            reshaped_data = []
            for row in data:
                if len(row) == 7:
                    reshaped_data.append(row)
                else:
                    print(f"Warning: Row does not have 7 values, skipping: {row}")

            # If valid reshaped data exists, convert it into a NumPy array
            if reshaped_data:
                reshaped_data = np.array(reshaped_data, dtype=float)  # Convert to float type

                # Check that reshaped data has enough rows as expected
                if reshaped_data.shape[0] < total_rows:
                    raise ValueError(f"Insufficient data: expected {total_rows}, but found {reshaped_data.shape[0]}")

                # Append the reshaped data and labels
                all_data.append(reshaped_data)
                all_labels.append([label] * reshaped_data.shape[0])  # Append the label for each data row

                # Print the shape of reshaped data for debugging
                print(f"Shape of data from file {file_path}: {reshaped_data.shape}")

        except Exception as e:
            # If any error occurs, display the error and continue processing the next file
            print(f"Error processing file {file_path}: {e}")
            continue

    # Raise an error if no valid data was processed
    if not all_data or not all_labels:
        raise ValueError("No valid data found in the directory")

    # Print the shapes of individual data entries for debugging
    print("Shapes of all_data before concatenation:")
    for i, data_array in enumerate(all_data):
        print(f"Data array {i} shape: {data_array.shape}")

    # Concatenate all data arrays along the sample axis (rows)
    try:
        all_data = np.concatenate(all_data, axis=0)  # Concatenate data arrays
    except Exception as e:
        print(f"Error during concatenation: {e}")

    # Concatenate labels
    all_labels = np.concatenate(all_labels, axis=0)

    # Return the processed data and corresponding labels
    return all_data, all_labels

# Importing all files on the Directory
data, labels = load_emothaw_data(r'../test/label copy/')  # Directory path

Processing file: ../test/label copy\0_hw00001.svc
Shape of data from file ../test/label copy\0_hw00001.svc: (2643, 7)
Processing file: ../test/label copy\0_hw00002.svc
Shape of data from file ../test/label copy\0_hw00002.svc: (5691, 7)
Processing file: ../test/label copy\0_hw00003.svc
Shape of data from file ../test/label copy\0_hw00003.svc: (3863, 7)
Processing file: ../test/label copy\0_hw00004.svc
Shape of data from file ../test/label copy\0_hw00004.svc: (1178, 7)
Processing file: ../test/label copy\0_hw00005.svc
Shape of data from file ../test/label copy\0_hw00005.svc: (770, 7)
Processing file: ../test/label copy\0_hw00006.svc
Shape of data from file ../test/label copy\0_hw00006.svc: (4051, 7)
Processing file: ../test/label copy\0_hw00007.svc
Shape of data from file ../test/label copy\0_hw00007.svc: (3050, 7)
Processing file: ../test/label copy\1_hw00001.svc
Shape of data from file ../test/label copy\1_hw00001.svc: (2560, 7)
Processing file: ../test/label copy\1_hw00002.svc
Shape o

## FEATURE EXTRACTION

In [4]:
# Cell 3
# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')
logger = logging.getLogger()

def extract_time_domain_features(data):
    logger.info("Starting time-domain feature extraction")
    start_time = datetime.now()
    
    # Vectorized computation of statistics across all samples at once
    # Compute stats for each feature across all samples
    means = np.mean(data, axis=1)
    stds = np.std(data, axis=1)
    mins = np.min(data, axis=1)
    maxs = np.max(data, axis=1)
    percentile_25 = np.percentile(data, 25, axis=1)
    percentile_50 = np.percentile(data, 50, axis=1)
    percentile_75 = np.percentile(data, 75, axis=1)
    
    # Stack all features horizontally for each sample
    time_features = np.column_stack([
        means, stds, mins, maxs,
        percentile_25, percentile_50, percentile_75
    ])
    
    end_time = datetime.now()
    elapsed_time = end_time - start_time
    logger.info(f"Feature extraction completed in {elapsed_time}")
    
    return time_features

def extract_frequency_domain_features(data):
    logger.info("Starting frequency-domain feature extraction")
    start_time = datetime.now()
    
    # Apply FFT to all samples at once
    freq_features = np.fft.fft(data, axis=1)
    freq_magnitude = np.abs(freq_features)
    
    # Calculate dominant frequencies and energy
    dominant_freqs = np.argmax(freq_magnitude, axis=1)
    freq_energies = np.sum(freq_magnitude, axis=1)
    
    # Combine features
    freq_features = np.column_stack([dominant_freqs, freq_energies])
    
    end_time = datetime.now()
    elapsed_time = end_time - start_time
    logger.info(f"Frequency-domain feature extraction completed in {elapsed_time}")
    
    return freq_features

def extract_statistical_features(data):
    logger.info("Starting statistical feature extraction")
    start_time = datetime.now()
    
    # Compute statistics for all samples at once
    means = np.mean(data, axis=1)
    medians = np.median(data, axis=1)
    variances = np.var(data, axis=1)
    
    # Compute skewness and kurtosis along the feature axis
    skewness = skew(data, axis=1)
    kurtosis_vals = kurtosis(data, axis=1)
    
    # Combine all statistical features
    statistical_features = np.column_stack([
        means, medians, variances, skewness, kurtosis_vals
    ])
    
    end_time = datetime.now()
    elapsed_time = end_time - start_time
    logger.info(f"Statistical feature extraction completed in {elapsed_time}")
    
    return statistical_features

# Example usage:
def process_features(data):
    """
    Process all features at once and combine them
    """
    time_features = extract_time_domain_features(data)
    freq_features = extract_frequency_domain_features(data)
    stat_features = extract_statistical_features(data)
    
    # Combine all features
    combined_features = np.concatenate([
        time_features,
        freq_features,
        stat_features
    ], axis=1)
    
    return combined_features

In [5]:
# Cell 4
# Extract all features
time_domain_features = extract_time_domain_features(data)
frequency_domain_features = extract_frequency_domain_features(data)
statistical_features = extract_statistical_features(data)

# Combine features
features = np.concatenate((time_domain_features, frequency_domain_features, statistical_features), axis=1)

print("Time Domain Features")
print(time_domain_features)
print("Frequency")
print(frequency_domain_features)
print("Statistical Features")
print(statistical_features)
print(time_domain_features.shape)
print(frequency_domain_features.shape)
print(statistical_features.shape)



2024-10-25 22:12:19,951 - Starting time-domain feature extraction
2024-10-25 22:12:20,632 - Feature extraction completed in 0:00:00.679939
2024-10-25 22:12:20,636 - Starting frequency-domain feature extraction
2024-10-25 22:12:20,708 - Frequency-domain feature extraction completed in 0:00:00.069294
2024-10-25 22:12:20,708 - Starting statistical feature extraction
2024-10-25 22:12:20,932 - Statistical feature extraction completed in 0:00:00.223948


Time Domain Features
[[3.25247000e+05 7.62450052e+05 1.00000000e+00 ... 3.04000000e+02
  1.87000000e+03 4.09560000e+04]
 [3.25264143e+05 7.62445522e+05 1.00000000e+00 ... 3.64500000e+02
  1.87000000e+03 4.09520000e+04]
 [3.25276571e+05 7.62443530e+05 1.00000000e+00 ... 4.02500000e+02
  1.87000000e+03 4.09535000e+04]
 ...
 [2.84505000e+05 6.87842004e+05 1.00000000e+00 ... 4.71000000e+02
  2.27000000e+03 9.49400000e+03]
 [2.84490429e+05 6.87850886e+05 1.00000000e+00 ... 4.17500000e+02
  2.27000000e+03 9.49300000e+03]
 [2.84473571e+05 6.87861127e+05 1.00000000e+00 ... 3.54500000e+02
  2.27000000e+03 9.49300000e+03]]
Frequency
[[       0.         15348978.31224078]
 [       0.         15349026.54532406]
 [       0.         15349082.72993092]
 ...
 [       0.         13785472.94690738]
 [       0.         13785522.00106619]
 [       0.         13785578.11577861]]
Statistical Features
[[3.25247000e+05 1.87000000e+03 5.81330082e+11 2.03885544e+00
  2.16142644e+00]
 [3.25264143e+05 1.87000000e

In [6]:
# Cell 5
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.saving import register_keras_serializable
from tensorflow.keras.regularizers import l2
@register_keras_serializable()
class MultiHeadSelfAttention(layers.Layer):
    def __init__(self, embed_dim, num_heads, dropout_rate=0.1, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.dropout_rate = dropout_rate
        if embed_dim % num_heads != 0:
            raise ValueError(f"embedding dimension = {embed_dim} should be divisible by number of heads = {num_heads}")
        self.projection_dim = embed_dim // num_heads
        self.query_dense = layers.Dense(embed_dim, kernel_regularizer=l2(0.01))
        self.key_dense = layers.Dense(embed_dim, kernel_regularizer=l2(0.01))
        self.value_dense = layers.Dense(embed_dim, kernel_regularizer=l2(0.01))
        self.combine_heads = layers.Dense(embed_dim, kernel_regularizer=l2(0.01))
        self.dropout = layers.Dropout(dropout_rate)
    
    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dropout_rate": self.dropout_rate,
        })
        return config

    def attention(self, query, key, value):
        score = tf.matmul(query, key, transpose_b=True)
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_score = score / tf.math.sqrt(dim_key)
        weights = tf.nn.softmax(scaled_score, axis=-1)
        weights = self.dropout(weights)
        output = tf.matmul(weights, value)
        return output, weights

    def separate_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs, training=False):
        batch_size = tf.shape(inputs)[0]
        query = self.query_dense(inputs)
        key = self.key_dense(inputs)
        value = self.value_dense(inputs)
        query = self.separate_heads(query, batch_size)
        key = self.separate_heads(key, batch_size)
        value = self.separate_heads(value, batch_size)
        attention, weights = self.attention(query, key, value)
        attention = tf.transpose(attention, perm=[0, 2, 1, 3])
        concat_attention = tf.reshape(attention, (batch_size, -1, self.embed_dim))
        output = self.combine_heads(concat_attention)
        return output

@register_keras_serializable()
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.2, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.ff_dim = ff_dim
        self.rate = rate
        self.att = MultiHeadSelfAttention(embed_dim, num_heads, rate)
        self.ffn = tf.keras.Sequential([
            layers.Dense(ff_dim, activation="relu", kernel_regularizer=l2(0.01)),
            layers.Dropout(rate),
            layers.Dense(embed_dim, kernel_regularizer=l2(0.01))
        ])
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "ff_dim": self.ff_dim,
            "rate": self.rate,
        })
        return config

    def call(self, inputs, training=False):
        attn_output = self.att(inputs, training=training)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

@register_keras_serializable()
class TransformerClassifier(Model):
    def __init__(self, num_classes, embed_dim, num_heads, ff_dim, num_layers, **kwargs):
        super().__init__(**kwargs)
        self.num_classes = num_classes
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.ff_dim = ff_dim
        self.num_layers = num_layers
        
        # Input normalization layer
        self.input_norm = layers.BatchNormalization()
        
        # Initial dense layer with dropout
        self.dense_input = layers.Dense(embed_dim, kernel_regularizer=l2(0.01))
        self.input_dropout = layers.Dropout(0.2)
        
        # Transformer blocks
        self.transformer_blocks = [
            TransformerBlock(embed_dim, num_heads, ff_dim) for _ in range(num_layers)
        ]
        
        # Output layers with regularization
        self.global_average_pooling = layers.GlobalAveragePooling1D()
        self.dropout1 = layers.Dropout(0.2)
        self.dense1 = layers.Dense(128, activation='relu', kernel_regularizer=l2(0.01))
        self.dropout2 = layers.Dropout(0.2)
        self.dense2 = layers.Dense(num_classes, activation='softmax', kernel_regularizer=l2(0.01))

    def get_config(self):
        config = super().get_config()
        config.update({
            "num_classes": self.num_classes,
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "ff_dim": self.ff_dim,
            "num_layers": self.num_layers,
        })
        return config

    def call(self, inputs, training=False):
        x = self.input_norm(inputs)
        x = self.dense_input(x)
        x = self.input_dropout(x, training=training)
        
        for transformer in self.transformer_blocks:
            x = transformer(x, training=training)
            
        x = self.global_average_pooling(x)
        x = self.dropout1(x, training=training)
        x = self.dense1(x)
        x = self.dropout2(x, training=training)
        return self.dense2(x)

## Transformer Block

Transformer Classifier

In [7]:
# Cell 6
import os
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)
import joblib
scaler_path = './saved_models/scaler.save'
joblib.dump(scaler, scaler_path)

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_scaled, labels, test_size=0.2, random_state=42, stratify=labels)

# Defining the model parameters
num_classes = 4
embed_dim = 64  # Increased embedding dimension
num_heads = 2    # Increased number of attention heads
ff_dim = 64    # Increased feedforward dimension
num_layers = 1   # Increased number of transformer layers

# Creating the Transformer model
model = TransformerClassifier(num_classes, embed_dim, num_heads, ff_dim, num_layers)

# Compile with modified learning rate and added metrics
model.compile(
    optimizer=Adam(learning_rate=0.001, clipnorm=1.0),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Training with early stopping and reduced learning rate on plateau
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=3,
    min_lr=0.00001
)

# Train the model
history = model.fit(
    X_train, 
    y_train,
    epochs=1,
    batch_size=256,
    validation_split=0.2,
    callbacks=[early_stopping, reduce_lr],
    class_weight={
        0: 1.0,  # Normal
        1: 1.0,  # Depression
        2: 1.0,  # Anxiety
        3: 1.0   # Stress
    }
)

# Save the model
save_directory = './saved_models/'
os.makedirs(save_directory, exist_ok=True)
model_path = os.path.join(save_directory, 'emotion_classifier.keras')
model.save(model_path)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")


195/195 ━━━━━━━━━━━━━━━━━━━━ 191s 869ms/step - accuracy: 0.7933 - loss: 4.0042 - val_accuracy: 0.9563 - val_loss: 0.9813 - learning_rate: 0.0010
487/487 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.9587 - loss: 0.9784
Test Loss: 0.9837498664855957, Test Accuracy: 0.9549399018287659


487/487 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step


In [8]:
# Cell 7
import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Step 1: Make predictions on the test set
y_pred_probs = model.predict(X_test)  # Predict probabilities for each class
y_pred = np.argmax(y_pred_probs, axis=1)  # Convert probabilities to class labels

# Step 2: Compute confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
class_labels = ['Normal','Depression', 'Anxiety', 'Stress']
# Step 3: Plot the confusion matrix using Seaborn for better visualization
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=class_labels, yticklabels=class_labels)
plt.ylabel('Actual Labels')
plt.xlabel('Predicted Labels')
plt.title('Confusion Matrix')
plt.show()


In [9]:
# Cell 8
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from tabulate import tabulate

# Print classification report with improved formatting
print("Classification Report:")
report = classification_report(y_test, y_pred, target_names=class_labels, output_dict=True)

# Convert classification report to a table
headers = ["Class", "Precision", "Recall", "F1-Score", "Support"]
rows = [[cls, 
         f"{metrics['precision']:.2f}", 
         f"{metrics['recall']:.2f}", 
         f"{metrics['f1-score']:.2f}", 
         int(metrics['support'])] for cls, metrics in report.items() if cls != 'accuracy']

# Add accuracy row separately
accuracy_row = ["Accuracy", "-", "-", f"{accuracy_score(y_test, y_pred) * 100:.2f}%", "-"]
rows.append(accuracy_row)

# Print the table
print(tabulate(rows, headers, tablefmt="fancy_grid"))

# Compute and display individual scores
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Display overall metrics in a formatted style
print("\nOverall Metrics:")
overall_metrics = [
    ["Metric", "Score"],
    ["Accuracy", f"{accuracy * 100:.2f}%"],
    ["Precision (weighted)", f"{precision:.2f}"],
    ["Recall (weighted)", f"{recall:.2f}"],
    ["F1-Score (weighted)", f"{f1:.2f}"]
]

print(tabulate(overall_metrics, headers="firstrow", tablefmt="fancy_grid"))


Classification Report:
╒══════════════╤═════════════╤══════════╤════════════╤═══════════╕
│ Class        │ Precision   │ Recall   │ F1-Score   │ Support   │
╞══════════════╪═════════════╪══════════╪════════════╪═══════════╡
│ Normal       │ 0.86        │ 1.00     │ 0.92       │ 4249      │
├──────────────┼─────────────┼──────────┼────────────┼───────────┤
│ Depression   │ 1.00        │ 1.00     │ 1.00       │ 3077      │
├──────────────┼─────────────┼──────────┼────────────┼───────────┤
│ Anxiety      │ 1.00        │ 1.00     │ 1.00       │ 3192      │
├──────────────┼─────────────┼──────────┼────────────┼───────────┤
│ Stress       │ 1.00        │ 0.86     │ 0.93       │ 5039      │
├──────────────┼─────────────┼──────────┼────────────┼───────────┤
│ macro avg    │ 0.96        │ 0.97     │ 0.96       │ 15557     │
├──────────────┼─────────────┼──────────┼────────────┼───────────┤
│ weighted avg │ 0.96        │ 0.95     │ 0.95       │ 15557     │
├──────────────┼─────────────┼─────────

In [10]:
# New Cell 9 - Classifier using saved model
import numpy as np
import tensorflow as tf
import os
from tensorflow.keras.models import load_model

def load_and_preprocess_file(file_path):
    """
    Load and preprocess a single file for classification
    """
    try:
        # Read the file
        with open(file_path, 'r') as f:
            lines = [line.strip() for line in f.readlines() if line.strip()]

        # Convert to DataFrame
        df = pd.DataFrame([line.split() for line in lines])
        
        # Skip empty files
        if df.empty:
            raise ValueError(f"File is empty after processing")

        # First row contains total number of rows
        total_rows = int(df.iloc[0, 0])
        
        # Extract feature data
        data = df.iloc[1:, :].values
        
        # Ensure each row has 7 features
        reshaped_data = []
        for row in data:
            if len(row) == 7:
                reshaped_data.append(row)
        
        if not reshaped_data:
            raise ValueError("No valid data rows found")
        
        # Convert to numpy array
        reshaped_data = np.array(reshaped_data, dtype=float)
        
        # Extract features
        time_features = extract_time_domain_features(reshaped_data)
        freq_features = extract_frequency_domain_features(reshaped_data)
        stat_features = extract_statistical_features(reshaped_data)
        
        # Combine features
        combined_features = np.concatenate((time_features, freq_features, stat_features), axis=1)
        combined_features = scaler.transform(combined_features)
        return combined_features
        
    except Exception as e:
        raise Exception(f"Error processing file {file_path}: {e}")

def classify_emotion(model_path, scaler_path, file_path):
    """
    Classify emotion using saved model and scaler and return confidences for all emotions
    """
    try:
        # Load the saved model and scaler
        model = load_model(model_path)
        scaler = joblib.load(scaler_path)
        
        # Process the input file
        features = load_and_preprocess_file(file_path, scaler)
        
        # Make prediction
        prediction = model.predict(features)
        
        # Calculate mean probabilities across all sequences
        mean_probabilities = np.mean(prediction, axis=0)
        
        # Map class indices to labels
        class_labels = ['Normal', 'Depression', 'Anxiety', 'Stress']
        
        # Create sorted list of (emotion, confidence) tuples
        emotion_confidences = [
            (label, float(prob * 100))
            for label, prob in zip(class_labels, mean_probabilities)
        ]
        # Sort by confidence in descending order
        emotion_confidences.sort(key=lambda x: x[1], reverse=True)
        
        # The predicted emotion is the first one (highest confidence)
        predicted_emotion = emotion_confidences[0][0]
        
        return predicted_emotion, emotion_confidences
        
    except Exception as e:
        raise Exception(f"Classification error: {e}")

def classify_directory(model_path, scaler_path, directory_path):
    """
    Classify all .svc files in a directory
    """
    results = []
    
    # Find all .svc files
    file_paths = glob.glob(os.path.join(directory_path, "*.svc"))
    
    if not file_paths:
        raise ValueError("No .svc files found in the directory")
    
    for file_path in file_paths:
        try:
            predicted_emotion, emotion_confidences = classify_emotion(
                model_path, scaler_path, file_path
            )
            results.append({
                'file': os.path.basename(file_path),
                'predicted_emotion': predicted_emotion,
                'confidences': emotion_confidences
            })
        except Exception as e:
            print(f"Error processing {file_path}: {e}")
            continue
    
    return results

# Example usage:
if __name__ == "__main__":
    # Path to your saved model
    model_path = './saved_models/emotion_classifier.keras'
    
    # Example 1: Classify a single file
    # try:
    #     file_path = './input_files/u00075s00001_hw00002.svc'
    #     emotion, confidence = classify_emotion(model_path, file_path)
    #     print(f"\nFile: {os.path.basename(file_path)}")
    #     print(f"Predicted Emotion: {emotion}")
    #     print(f"Confidence: {confidence:.2f}%")
    # except Exception as e:
    #     print(f"Error classifying single file: {e}")
    
    # Example 2: Classify all files in a directory
    try:
        directory_path = './input_files/'
        # directory_path = '../test/label copy/'
        results = classify_directory(model_path, directory_path)
        
        print("\nDirectory Classification Results:")
        print("================================")
        for result in results:
            print(f"\nFile: {result['file']}")
            print(f"Predicted Emotion: {result['predicted_emotion']}")
            print(f"Confidence: {result['confidence']:.2f}%")
    except Exception as e:
        print(f"Error classifying directory: {e}")



2024-10-25 22:16:19,025 - From c:\Users\katod\anaconda3\Lib\site-packages\keras\src\backend\tensorflow\core.py:204: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

2024-10-25 22:16:20,151 - Starting time-domain feature extraction
2024-10-25 22:16:20,151 - Feature extraction completed in 0:00:00
2024-10-25 22:16:20,151 - Starting frequency-domain feature extraction
2024-10-25 22:16:20,167 - Frequency-domain feature extraction completed in 0:00:00
2024-10-25 22:16:20,167 - Starting statistical feature extraction
2024-10-25 22:16:20,182 - Statistical feature extraction completed in 0:00:00.015620


83/83 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step


2024-10-25 22:16:25,489 - Starting time-domain feature extraction
2024-10-25 22:16:25,507 - Feature extraction completed in 0:00:00.010090
2024-10-25 22:16:25,509 - Starting frequency-domain feature extraction
2024-10-25 22:16:25,509 - Frequency-domain feature extraction completed in 0:00:00
2024-10-25 22:16:25,523 - Starting statistical feature extraction
2024-10-25 22:16:25,548 - Statistical feature extraction completed in 0:00:00.018185


80/80 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step


2024-10-25 22:16:31,219 - Starting time-domain feature extraction
2024-10-25 22:16:31,232 - Feature extraction completed in 0:00:00.011041
2024-10-25 22:16:31,236 - Starting frequency-domain feature extraction
2024-10-25 22:16:31,242 - Frequency-domain feature extraction completed in 0:00:00
2024-10-25 22:16:31,249 - Starting statistical feature extraction
2024-10-25 22:16:31,305 - Statistical feature extraction completed in 0:00:00.052578


97/97 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step


2024-10-25 22:16:41,296 - Starting time-domain feature extraction
2024-10-25 22:16:41,309 - Feature extraction completed in 0:00:00.012933
2024-10-25 22:16:41,312 - Starting frequency-domain feature extraction
2024-10-25 22:16:41,317 - Frequency-domain feature extraction completed in 0:00:00.002560
2024-10-25 22:16:41,321 - Starting statistical feature extraction
2024-10-25 22:16:41,361 - Statistical feature extraction completed in 0:00:00.034596


86/86 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step


2024-10-25 22:16:47,302 - Starting time-domain feature extraction
2024-10-25 22:16:47,323 - Feature extraction completed in 0:00:00.011275
2024-10-25 22:16:47,326 - Starting frequency-domain feature extraction
2024-10-25 22:16:47,339 - Frequency-domain feature extraction completed in 0:00:00.001997
2024-10-25 22:16:47,342 - Starting statistical feature extraction
2024-10-25 22:16:47,388 - Statistical feature extraction completed in 0:00:00.025883


177/177 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step


2024-10-25 22:16:55,125 - Starting time-domain feature extraction
2024-10-25 22:16:55,129 - Feature extraction completed in 0:00:00.001998
2024-10-25 22:16:55,132 - Starting frequency-domain feature extraction
2024-10-25 22:16:55,139 - Frequency-domain feature extraction completed in 0:00:00
2024-10-25 22:16:55,139 - Starting statistical feature extraction
2024-10-25 22:16:55,139 - Statistical feature extraction completed in 0:00:00


26/26 ━━━━━━━━━━━━━━━━━━━━ 6s 116ms/step


2024-10-25 22:17:05,807 - Starting time-domain feature extraction
2024-10-25 22:17:05,837 - Feature extraction completed in 0:00:00.027824
2024-10-25 22:17:05,839 - Starting frequency-domain feature extraction
2024-10-25 22:17:05,848 - Frequency-domain feature extraction completed in 0:00:00.002996
2024-10-25 22:17:05,863 - Starting statistical feature extraction
2024-10-25 22:17:05,954 - Statistical feature extraction completed in 0:00:00.090552


193/193 ━━━━━━━━━━━━━━━━━━━━ 9s 41ms/step

Directory Classification Results:

File: 0_hw00001.svc
Predicted Emotion: Normal
Confidence: 99.74%

File: 1_hw00001.svc
Predicted Emotion: Depression
Confidence: 99.46%

File: 2_hw00003.svc
Predicted Emotion: Anxiety
Confidence: 99.82%

File: 3_synthetichw00006.svc
Predicted Emotion: Anxiety
Confidence: 99.90%

File: hw00002.svc
Predicted Emotion: Stress
Confidence: 99.81%

File: u00067s00001_hw00005.svc
Predicted Emotion: Depression
Confidence: 99.60%

File: u00075s00001_hw00002.svc
Predicted Emotion: Stress
Confidence: 97.46%


# END
